# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

Возьмем данные с соревнования [Dialog Evaluation 2016](http://www.dialog-21.ru/evaluation/2016/spelling_correction/) по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Я удалили из данных случаи, когда в словах пропущен или вставлен пробел, чтобы было проще сопоставить слова в предложении. 

In [55]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [56]:
len(true)

915

In [57]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [67]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [68]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [69]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [70]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.12886443221610805


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [71]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Тексты из википедии для этого хорошо подходят.

In [72]:
corpus = open('wiki_data.txt', encoding='utf8').read()

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [178]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))


In [195]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [196]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0 
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка 
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))
    

In [197]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred, ))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      8707
           1       0.59      0.88      0.71      1288

    accuracy                           0.91      9995
   macro avg       0.79      0.90      0.83      9995
weighted avg       0.93      0.91      0.91      9995



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига. Идея алгоритма очень простая - для каждого неправильного слова нужно сгенерировать варианты исправлений и выбрать из них тот , что есть в словаре, а если таких несколько, то выбрать наиболее вероятный. 

Неправильными считаются слова, которых нет в словаре (также как в функции выше). А вероятность слова расчитывается по формуле - абсолютная частота слова в корпусе разделить на количество слов в корпусе.

Абсолютные частоты лежат в счетчике

In [198]:
vocab.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

В вероятности они преобразуются вот такой функцией

In [203]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N


#### Отвлечемся на вероятности

У нас есть три основных способа посмотреть на конкретную вероятность:

1) в питоне по умолчанию используется запись маленьких и больших чисел через число e в степени. Степень выводится сразу после числа e (если в начале степени стоит - , то это число меньше единицы). Чем больше/меньше степень, тем больше/меньше число

In [262]:
print(P('солнце')) # вероятность слова солнце по нашему словарю

2.4440966240624417e-05


Также можно самому задавать числа таким образом - это может сэкономить немного времени и сократить код. Так часто задают learning_rate в нейронных сетях 

In [217]:
1e1, 1e-1, 1e-4, 1e4

(10.0, 0.1, 0.0001, 10000.0)

2) для того, чтобы вывести привычную десятичную дробь, можно воспользоваться строковым методом .format с указанием количества знаков после запятой

In [263]:
print('{0:.15f}'.format(P('солнце')))

0.000024440966241


3) Вероятности слов обычно очень маленькие. С ними неудобно работать, а в расчетах с умножением вероятностей это вообще может привести к занулению (underflow). Поэтому часто используется логарифм вероятности (основание не имеет значения).  

In [239]:
print(np.log(P('солнце')))

-10.61924988922861


#### Несколько фактов про логарифмы вероятностей

Вероятность это число от 0 до 1, поэтому её логарифм всегда будет от минус бесконечности до 0. 

In [243]:
np.log(0.0001), np.log(0.999), np.log(1)

(-9.210340371976182, -0.0010005003335835344, 0.0)

Логарифм от нуля равен бесконечности. Бесконечность сломает вычисления, в которых она окажется слагаемым

In [252]:
np.log(0) + np.log(1e100)

/Users/mnefedov/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

Есть специальная функция в numpy, которая добавляет к вероятности единицу прежде чем брать логарифм. Важное уточнение: нужно преобразовывать вероятности каким-то одним способом - либо np.log либо np.log1p, нельзя для нуля делать np.log1p, а для всех остальных np.log!

In [264]:
np.log1p(0), np.log1p(1)

(0.0, 0.6931471805599453)

Для того, чтобы вернуться к обычной вероятности нужно взять экспоненту от логарифма вероятности. 

In [265]:
np.exp(np.log(0.1))

0.10000000000000002

Если вы использовали np.log1p, то вернутся поможет np.expm1 (она вычитает единицу после экспоненциирования)

In [266]:
np.expm1(np.log1p(0.1)), np.expm1(np.log1p(0.0))

(0.1, 0.0)

Умножение вероятностей тождественно сложению логарифмов вероятностей

In [274]:
(0.01*0.02)

0.0002

In [275]:
np.exp(np.log(0.01) + np.log(0.02)) 
# о том почему результат выглядит так странно можно почитать 
# вот тут - https://docs.python.org/3/tutorial/floatingpoint.html

0.0002000000000000002

#### Вернемся к нашей задаче

Теперь самое интересное - способ генерации вариантов исправлений. Они генерируются 4 типами эвристик: удаление, перестановка, замена, вставка. 

1) здаление - по очереди выбрасываем из слова 1 букву (слово - лово, сово, слво, слоо, слов)  
2) перестановка - по очереди меняем соседние буквы (слово - лсово, солво, слвоо, слоов)  
3) замена - по очереди заменям каждую букву на другую букву алфавита (слово - алово, блово, влово, глово...)  
4) вставка - по очереди вставляем между соседними буквами букву алфавита (слово - салово, сблово, свлово, сглово...)  

В алгоритма два уровня генерации - сначала генерируются варианты для оригинального слова, а потом варианты для каждого варианта. Таким образом, максимальное допустимое отличие для ошибки и предсказания - 2 буквы.

In [278]:
# оригинальный код вот тут - 
# я только адаптировал его под русский язык

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Попробуем исправить

In [284]:
%%time
correction('сонлце')

CPU times: user 301 µs, sys: 0 ns, total: 301 µs
Wall time: 305 µs


'солнце'

In [285]:
%%time
correction('опофеоз')

CPU times: user 352 µs, sys: 0 ns, total: 352 µs
Wall time: 356 µs


'апофеоз'

In [286]:
word = 'сонце'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

Выводов по единичным примерам не сделаешь, поэтому давайте запустим на всем нашем корпусе

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [288]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


Получается, что в целом не стало лучше. Хотя 50% опечаток исправляются корректно

In [290]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.870735367683842
0.5108695652173914
0.07603077983231882


Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [292]:
%%time
correction('солнвце')

CPU times: user 1.28 ms, sys: 8.99 ms, total: 10.3 ms
Wall time: 17.9 ms


'солнце'

In [293]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 1.98 s, sys: 25.4 ms, total: 2.01 s
Wall time: 2.02 s


'насмехатьсяаававттававаываываы'

Посмотрим, как исправляются самые частотные ошибки.

In [294]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'седая'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'факто'),
 ('очень', 'оооочень', 'сорочень'),
 ('это', 'ето', 'что')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования

Самое известное расстояние редактирования - расстояние Левенштейна. Тут мы не будет поднобно разбирать алгоритм, можете почитать [тут](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0), посмотреть более понятный разбор [тут](https://www.youtube.com/watch?v=MiqoA-yF-0M), а код на питоне есть [тут](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0).
Про самого Левенштейна можно почитать вот тут - https://nplus1.ru/material/2017/09/25/vladimir-levenshtein

Основная идея - найти минимальное число исправлений, которое нужно сделать в слове А, чтобы получить слово Б. Причем допустимы только три вида исправлений - удаление, вставка, замена. 

Ещё есть расстояние Дамерау-Левенштейна - почти то же самое, только разрешена ещё операция перестановки.

Есть библиотека textdistance, в которой реализованы многие методы нахождения расстояний.

In [132]:
!pip install textdistance

You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.7.2/bin/python3.7 -m pip install --upgrade pip' command.


In [297]:
import textdistance

In [310]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [299]:
%%time
get_closest_match_with_metric('сонце', vocab, 3, textdistance.hamming)

CPU times: user 5.74 s, sys: 45 ms, total: 5.78 s
Wall time: 5.81 s


[('конце', 0.8), ('монце', 0.8), ('сонче', 0.8)]

In [300]:
%%time
get_closest_match_with_metric('опофеоз', vocab, 5, textdistance.damerau_levenshtein)

CPU times: user 40.6 s, sys: 107 ms, total: 40.7 s
Wall time: 40.8 s


[('апофеоз', 0.8571428571428572),
 ('апофеоза', 0.75),
 ('апофеозом', 0.6666666666666667),
 ('апофеты', 0.5714285714285714),
 ('опорной', 0.5714285714285714)]

In [311]:
%%time
get_closest_match_with_metric('кул', vocab, 5, textdistance.damerau_levenshtein)

CPU times: user 102 µs, sys: 1 µs, total: 103 µs
Wall time: 106 µs


[('баба', 0.75)]

Можно немного ускорить поиск, сократив количество слов в словаре. Возьмем только те, что встречаются больше 5 раз.

In [306]:
vocab_top = {word:count for word, count in vocab.items() if count > 5}

In [307]:
%%time
get_closest_match_with_metric('кул', vocab_top, 5, textdistance.damerau_levenshtein)

CPU times: user 3.23 s, sys: 3.76 ms, total: 3.23 s
Wall time: 3.23 s


[('кул', 1.0), ('акул', 0.75), ('коул', 0.75), ('куль', 0.75), ('кули', 0.75)]

Сильно быстрее не стало

Еще в питоне есть встроенная библиотека для нахождения близких строк

In [308]:
from difflib import get_close_matches

In [309]:
%%time
get_close_matches('опофеоз', vocab_top.keys(), n=1)

CPU times: user 273 ms, sys: 1.91 ms, total: 274 ms
Wall time: 273 ms


['пропофол']

Работает тоже не очень быстро.

С большим словарем даже оптимизированные версии будут работать очень долго. И принципиально с этим ничего не сделаешь, т.к. поиск минимального расстояния редактирования - сложный алгоритм, а хороший словарь должен быть большим. 

Но можно обратить внимание на то, что расстояние редактирование до большинства слов в словаре считать бессмысленно, т.к. они очевидно отличаются слишком сильно (например, слово "лес" и "заноза" настолько разные, что нам неважно какое между ними расстояние редактирования). Нам нужно придумать способ, который бы позволил нам выделить из словаря только те слова, до которых имеет смысл считать расстояние редактирования. 

Кажется, что **косинусное расстояние по мешку символов** хорошо для этого подходит. Близкими будут слова, состоящие из одинаковых символов. Расстояние редактирования между такими словами может быть и большим (акула-лука),  и маленьким (акула-акул), поэтому их придется проверить расстоянием левенштейна прежде, чем предсказывать. При этом мы можем быть уверены, что далекими по косинусу точно не будут слова с маленьким расстоянием редактирования! И соответственно их можно сразу отбросить из кандидатов на исправление.

Косинусное расстояние работает на векторах, а векторные операции сильно быстрее любых циклов (а внутри расстояния левенштейна иммено циклы).

Сделаем поиск похожих по векторам символов, из которых состоит слово. Косинусное расстояние между векторами слов не равно расстоянию редактирования, т.к. в нем не учитывается порядок символов. Близкими будут слова, состоящие из одинаковых символов. Но мы можем быть уверены, что близкими точно не будут слова с большим косинсным расстоянием, и можем их не проверять их честной метрикой редактирования.

In [313]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [347]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [348]:
X.shape

(368802, 1000)

In [349]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [350]:
%%time
get_closest_match_vec('сонце', X, vec) # это расстояние - чем меньше тем лучше

CPU times: user 321 ms, sys: 47.9 ms, total: 369 ms
Wall time: 369 ms


[('солнце', 0.20000000000000007),
 ('сон', 0.22540333075851648),
 ('саксонцев', 0.2409278847234103),
 ('херсонцев', 0.258380151290434),
 ('соне', 0.26213521262737816),
 ('донце', 0.26213521262737816),
 ('солнцем', 0.26970325665977846),
 ('ньонце', 0.26970325665977846),
 ('сонм', 0.2828628343993639),
 ('бсон', 0.2828628343993639),
 ('соню', 0.2828628343993639),
 ('эсон', 0.2828628343993639),
 ('бессоннице', 0.2953357365823558),
 ('солнцев', 0.29835358455437655),
 ('конце', 0.30000000000000004),
 ('монце', 0.30000000000000004),
 ('нсон', 0.30000000000000004),
 ('сонче', 0.30000000000000004),
 ('кесон', 0.30000000000000004),
 ('бессонницей', 0.3184457989035222)]

Напишем функцию, которая принимает слово и находит ближайшее к нему в словаре с помощью косинусного расстояния, а затем проверяет варианты честной метрикой редактирования. 

In [351]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [352]:
%time
get_closest_hybrid_match('солнечный', X, vec)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


[('солнечный', 1.0),
 ('солнечные', 0.8888888888888888),
 ('солнечным', 0.8888888888888888)]

Оценим такой метод исправления.

In [355]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [356]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8527263631815908
0.4658385093167702
0.09004249454461927


К сожалению лучше не стало. Но возможно дело в том, что в алгоритме Норвига мы учитывали вероятности слов, а тут нет. Попробуйте добавить ранжирование равнозначных по расстоянию редактирования кандидатов по вероятности и оцените качество.

## Готовые инструменты

Есть несколько готовых опечаточников:  
1) Hunspell - https://pypi.org/project/hunspell/  
2) Jamspell - https://github.com/bakwc/JamSpell#python  
3) Яндекс.Спеллер - https://yandex.ru/dev/speller/ (только через API)


Если вам понадобится в серьезной задаче исправлять опечатки, то начните с них, а не с алгоритма Норвига.